In [8]:
API_KEY = 'sk-BwmSb3q5qK97zIJYW2rYpFXTn-8ua1kiuJy1wEKf0YT3BlbkFJCD7_y0woQTDGh2lauJaCQW5-WgeC1xiqcIiaUK3YAA'
URL = 'https://sites.google.com/site/maxfriedrichsteinhardt/home/research'

In [11]:
from bs4 import BeautifulSoup
from openai import OpenAI
import requests
from db_config import db_config



def extract_relevant_html(html_content):
    """
    Use BeautifulSoup to extract the relevant parts of the HTML that contain the publications.
    """
    soup = BeautifulSoup(html_content, 'html.parser')

    # Remove scripts and styles
    for element in soup(['script', 'style', 'header', 'footer', 'nav', 'aside']):
        element.decompose()

    # Attempt to find the main content area
    main_content = soup.body
        
    # Extract the text content
    text_content = main_content.get_text(separator='\n', strip=True)
    return text_content

def extract_publications(response_text):
    prompt = f"""
    You are an AI assistant that extracts academic publication details from HTML content.

    Extract all the publications from the following HTML content from {URL}. For each publication, provide:
    - Title
    - Authors
    - Year
    - Journal (e.g., journal or conference name)

    Provide the output as a JSON array of objects with the keys: "title", "authors", "year", "venue".
    HTML Content:
    {response_text}
    """


    client = OpenAI(
        # This is the default and can be omitted
        api_key= db_config['openai_api_key']
    )


    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        model="gpt-3.5-turbo",
    )
    
    response = chat_completion.choices[0].message.content

    # Attempt to parse the response as JSON
    try:
        publications = json.loads(response)
    except json.JSONDecodeError:
        print("Failed to parse JSON. Attempting to fix the response...")
        # Try to extract JSON content from the response using regex
        import re
        json_text_match = re.search(r'\[.*\]', response, re.DOTALL)
        if json_text_match:
            json_text = json_text_match.group(0)
            try:
                publications = json.loads(json_text)
            except json.JSONDecodeError:
                print("Failed to parse the extracted JSON text.")
                return None
        else:
            print("No JSON array found in the response.")
            return None
        
    return response

import json

html_content = requests.get(URL).text
text_content = extract_relevant_html(html_content)
publications = extract_publications(text_content)

In [ ]:
import json

html_content = requests.get(URL).text
text_content = extract_relevant_html(html_content)
publications = extract_publications(text_content)

In [15]:
papers = json.loads(publications)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [14]:
# Save publications as JSON
with open('publications.json', 'w', encoding='utf-8') as f:
    json.dump(publications, f, ensure_ascii=False, indent=4)

'```json\n[\n    {\n        "title": "The long run impact of childhood interracial contact on residential segregation",\n        "authors": ["Merlino, L.", "Steinhardt, M.", "Wren-Lewis. L."],\n        "year": "2014",\n        "venue": "Journal of Public Economics"\n    },\n    {\n        "title": "Women in Political Power and School Closure during COVID Times",\n        "authors": ["Danzer, N.", "Garcia-Torres, S.", "Steinhardt, M.", "Stella, L."],\n        "year": "2014",\n        "venue": "Economic Policy"\n    },\n    {\n        "title": "Opening Heaven\'s Door: Public Opinion and Congressional Votes on the 1965 Immigration Act",\n        "authors": ["Facchini, G.", "Hatton, T", "Steinhardt, M."],\n        "year": "2014",\n        "venue": "Journal of Economic History"\n    },\n    {\n        "title": "Immigration and New Firm Formation – Evidence from a Quasi-experimental Setting in Germany",\n        "authors": ["Jahn, V.", "Steinhardt, M."],\n        "year": "2023",\n        "ve